In [ ]:
pip install dune-client

In [1]:
from dotenv import load_dotenv

load_dotenv(".env")

True

In [ ]:
import os
from dune_client.client import DuneClient

DUNE_API_KEY=os.getenv("DUNE_API_KEY")
dune = DuneClient(DUNE_API_KEY)
query_result = dune.get_latest_result(3888983)
print(query_result.result)

In [ ]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

In [ ]:
query = QueryBase(
    name="ethereum_get_info_from_symbol",
    query_id=3893389,
    params=[
        QueryParameter.text_type(name="symbol", value="usdt"),
    ],
)
results_df = dune.run_query_dataframe(query)
print(results_df.to_json())

In [ ]:
from web3 import Web3

infura_url = f"https://mainnet.infura.io/v3/{os.getenv('INFURA_API_KEY')}"
web3 = Web3(Web3.HTTPProvider(infura_url))

def is_contract_address(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return code != b''

print(is_contract_address("0xC0ffeEBABE5D496B2DDE509f9fa189C25cF29671"))

In [ ]:
from dune_tools import top_10_eth_transaction

result=top_10_eth_transaction.invoke({"min_amout":1000})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address
import json

token_balances_str = get_token_balances_of_address.invoke(
    {"address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d"}
)
token_balances = json.loads(token_balances_str)
    # 生成Token余额分析报表
token_balance_report = []
for token in token_balances:
    balance_usd = token.get("balance_usd", "N/A")
    token_balance_report.append(
        f"Token: {token['token_symbol']} (Contract: {token['token_address']}, Liquidity on DEX in 3 months:${token['total_liquidity_usd']})\n"
        f"Balance: {token['balance']} (USD: {balance_usd} Price:${token['price']})\n"
    )

# Token余额分析部分
token_analysis = "\n".join(token_balance_report)
print(token_analysis)

In [ ]:
import requests
import time
import os

def get_address_tags_from_etherscan(addresses):
    ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")  # Replace with your actual Etherscan API key
    ETHERSCAN_API_URL = 'https://api.etherscan.io/api'
    
    address_tags = {}
    
    for address in addresses:
        params = {
            'module': 'account',
            'action': 'txlist',
            'address': address,
            'apikey': ETHERSCAN_API_KEY,
            'startblock': 0,
            'endblock': 99999999,
            'page': 1,
            'offset': 10,  # Example to limit query, adjust as necessary
            'sort': 'asc'
        }
        
        response = requests.get(ETHERSCAN_API_URL, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data['status'] == '1':
                # Extracting labels from the result
                address_tags[address] = data['result']
            else:
                address_tags[address] = 'No data found'
        else:
            address_tags[address] = f"Error: {response.status_code}"
        
        # Respect Etherscan API rate limits
        time.sleep(1)
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e", 
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D"
]
address_tags = get_address_tags_from_etherscan(addresses)
for address, tags in address_tags.items():
    print(f"Address: {address}, Tags: {tags}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

def get_address_tag_from_etherscan(address):
    """
    Get the tag of a single Ethereum address from Etherscan.
    """
    url = f"https://etherscan.io/address/{address}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        tag_element = soup.find("span", {"class": "u-label u-label--xs u-label--info rounded"})

        if tag_element:
            return tag_element.text.strip()
        else:
            return "No tag found"
    else:
        return f"Failed to retrieve page, status code: {response.status_code}"

def get_multiple_address_tags(addresses):
    """
    Get tags for multiple Ethereum addresses from Etherscan.
    """
    address_tags = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(get_address_tag_from_etherscan, addresses)
        
    for address, tag in zip(addresses, results):
        address_tags[address] = tag
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e",
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D",
    "0x5a52E96BACdAbDcf4b16a7F8cE7C6c31E3Cc4aF6"
]

address_tags = get_multiple_address_tags(addresses)
for address, tag in address_tags.items():
    print(f"Address: {address}, Tag: {tag}")

In [ ]:
from dune_tools import get_address_tag_from_etherscan

result=get_address_tag_from_etherscan.invoke({"address":"0xF838ac8921e8d4Efdd20e605C2855f20691ba6a3"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0xA0c68C638235ee32657e8f720a23ceC1bFc77C77"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0x59a19d8c652fa0284f44113d0ff9aba70bd46fb4"})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address,extract_main_token_of_balances

# balances=get_token_balances_of_address.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
# print(balances)
# result=extract_main_token_of_balances(balances)
# print(result)
from dune_tools import extract_token_from_balances
print(extract_token_from_balances(data=balances))

In [ ]:
from dune_tools import get_erc20_transfer_of

result=get_erc20_transfer_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d','contract_address':'0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'})
print(result)

In [ ]:
from dune_tools import get_eth_movements_of

result=get_eth_movements_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
print(result)

In [2]:
import pprint
from ethereum_address_analysis_agent import ethereum_address_analysis_agent_executor

# ethereum_address_analysis_agent_executor.invoke(
#     {
#         "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
#         "chat_history": [],
#     },
#     config={
#         "configurable": {"llm": "openai_gpt_4o"},
#     },
# )

chunks = []
async for chunk in ethereum_address_analysis_agent_executor.astream_events(
    {
        "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "chat_history": [],
    },
    version="v1",
    config={
        "configurable": {"llm": "openai_gpt_4o"},
    },
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

/Users/zhangleping/github.com/holynull/ethereum_address_analysis_agent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




> Entering new AgentExecutor chain...
------
{'data': {...},
 'event': 'on_chain_start',
 'metadata': {},
 'name': "Eddie's Assistant Agent",
 'run_id': 'f33102e4-2ee1-4a79-aa98-668cd6904ef9',
 'tags': []}
------
{'data': {},
 'event': 'on_chain_start',
 'metadata': {},
 'name': 'RunnableSequence',
 'run_id': 'eb494e9c-e893-4274-8479-cdcf2c0d07a7',
 'tags': []}
------
{'data': {},
 'event': 'on_chain_start',
 'metadata': {},
 'name': 'RunnableAssign<agent_scratchpad>',
 'run_id': '88a218f0-419a-43b2-b7bd-0da04203bd8c',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chain_stream',
 'metadata': {},
 'name': 'RunnableAssign<agent_scratchpad>',
 'run_id': '88a218f0-419a-43b2-b7bd-0da04203bd8c',
 'tags': [...]}
------
{'data': {},
 'event': 'on_chain_start',
 'metadata': {},
 'name': 'RunnableParallel<agent_scratchpad>',
 'run_id': '6e3e797e-68af-4d3c-9f42-78e65dc0c2a8',
 'tags': []}
------
{'data': {},
 'event': 'on_chain_start',
 'metadata': {},
 'name': 'RunnableLambda',
 'run_id

2024-07-29 10:02:30,210 INFO httpx HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': 'd95e6278-012c-4c21-b90d-4a07f7ba1725',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': 'd95e6278-012c-4c21-b90d-4a07f7ba1725',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': 'd95e6278-012c-4c21-b90d-4a07f7ba1725',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': 'd95e6278-012c-4c21-b90d-4a07f7ba1725',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': 'd95e6278-012c-4c21-b90d-4a07f7ba1725',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': 'd95e6278-012c-4c21-b90d-4a07f7ba1725',
 'tags': [...]

2024-07-29 10:02:30,606 INFO dune_client.api.base executing 3916915 on medium cluster
2024-07-29 10:02:30,606 INFO dune_client.api.base executing 3912735 on medium cluster
2024-07-29 10:02:30,607 INFO dune_client.api.base executing 3924884 on medium cluster


...

[{"address":"0xdfd5293d8e347dfe59e90efd55b2956a1343963d","labels":"[Binance User Binance 16 DEX Trader 2 years old DEX trader Number of DEXs traded on: 1 Sparse Trader Uniswap User $5k-$10k avg. DEX trade value][Binance 16]"}]------
{'data': {...},
 'event': 'on_tool_end',
 'metadata': {},
 'name': 'get_address_labels',
 'run_id': 'e01bcde0-3236-4c45-9fe6-0ea3a9a99d36',
 'tags': []}


2024-07-29 10:03:22,736 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING
2024-07-29 10:03:24,231 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING
2024-07-29 10:03:25,593 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING
2024-07-29 10:03:27,305 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING
2024-07-29 10:03:28,734 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING
2024-07-29 10:03:30,172 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING
2024-07-29 10:03:31,545 INFO dune_client.api.base waiting for query execution 01J3Y3K6Z70C897C7WDC1GWTEN to complete: ExecutionState.EXECUTING

{'high_price_token': [{'address': '0x6c6ee5e31d828de241282b9606c8e98ea48526e2', 'symbol': 'HOT'}, {'address': '0x607f4c5bb672230e8672085532f7e901544a7375', 'symbol': 'RLC'}, {'address': '0xdefa4e8a7bcba345f687a2f1456f5edd9ce97202', 'symbol': 'KNC'}, {'address': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'symbol': 'AAVE'}, {'address': '0x3472a5a71965499acd81997a54bba8d852c6e53d', 'symbol': 'BADGER'}], 'high_liquidity_token': [], 'balances': '## Balances of address 0xdfd5293d8e347dfe59e90efd55b2956a1343963d:\nToken: USDC\nContract address: 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48\nBalance: 8.4437625499902e+07 (USD: 8.445358421112147e+07 Price:$1.000189)\nToken: <nil>\nContract address: 0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee\nBalance: 11087.1408472216 (USD: 3.642458382537713e+07 Price:$3285.3)\nToken: FDUSD\nContract address: 0xc5f0f7b66764f6ec8c8dff7ba683102295e16409\nBalance: 3.5672508129039e+07 (USD: 3.565213912689731e+07 Price:$0.999429)\nToken: USDT\nContract address: 0xdac1

2024-07-29 10:03:40,120 INFO httpx HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': '8b8fbd87-cf1b-4b08-ad09-cb4d790b7166',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': '8b8fbd87-cf1b-4b08-ad09-cb4d790b7166',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': '8b8fbd87-cf1b-4b08-ad09-cb4d790b7166',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': '8b8fbd87-cf1b-4b08-ad09-cb4d790b7166',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': '8b8fbd87-cf1b-4b08-ad09-cb4d790b7166',
 'tags': [...]}
------
{'data': {...},
 'event': 'on_chat_model_stream',
 'metadata': {...},
 'name': 'ChatOpenAI',
 'run_id': '8b8fbd87-cf1b-4b08-ad09-cb4d790b7166',
 'tags': [...]